In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

import topiary

from Bio.Align.Applications import MuscleCommandline

## 0. Project organization

Keeping track of all of the files is always annoying in these projects.  I'm not sure there is a "right" way to do it, but here are a few principles that have kept me sane over the years:

+ Write up a `README` file and save it in the directory where you do your work.  This should say what you do at each step, the date you did it, and what files were involved.  This is the lab notebook for the project.  I prefer to keep this as a standalone file rather than a jupyter notebook because a text file will *always* be readable; who knows how jupyter will change in the future.
+ Whenever you have to save out a file, do it with a numbered prefix and description.  Something like `00_initial-sequence-to-blast.fasta`, `01_blast-results.xml`, etc. This way, if you sort the directory, you can see what you did, in what order. 
+ Keep your sequences in a `.csv` file with columns for species, database id, raw sequence, aligned sequence, etc. You can then share this `.csv` file as the supplement to your paper.  *This protocol implements this `.csv` strategy throughout.*
+ One of the standard phylogenetics file formats (`.phy`) requires sequences have 10 character names. This is *very annoying.* To deal with this problem, the pipeline assigns every sequence you add a unique 10-character ID of the format `XXXXXXXXXX` where `n` indicates an integer. At various points in the pipeline, you can interconvert between these internal sequence names and various human-readable names. 

## 1. Create a set of BLAST .xml files containing sequences

Building sequence datasets is complicated.  How you go about it depends on what your goals are for the project.  In this protocol, I am going to assume we are looking members of protein families (meaning a mixture of orthologs and paralogs) from animals (meaning minimal lateral gene transfer). 

I'll mention two questions that often come up.  The first is: *should I include paralogs and orthologs?* For a gene tree—what we normally generate—you generally want to a large number of orthologs and paralogs. A good check for the quality of the tree is whether the orthologs group together and roughly reproduce the species tree.  In practice, this means BLASTing away and then grabbing as many sequences as possible without worrying too much about whether you are pulling down orthologs or paralogs.  

One reason this is useful is that sequence databases sometimes have paralogs and orthologs mislabeled.  Imagine you are building a tree of protein *X*.  You BLAST the NCBI and pull out out a sequence labeled protein *Y*.  You then build a tree, including this protein, and find it groups squarely with the *X* proteins, but not the other *Y* proteins in your dataset.  The simplest explanation for this result is that the protein was annotated incorrectly.   If you had dropped this sequence from your analysis, just because it was labeled *Y*, you would have removed valuable information from your dataset.  

A second question: *should I include "hypothetical," predicted", and "low-quality" sequences?* As is usual for bioinformatics, the answer is "it depends." If the protein has a huge number of isoforms and weirdo splice sites, the predicted genes could be difficult to align and thus mess up your phylogeny. On the other hand, if the protein has a very well defined set of exons, you'd probably be okay. Further, you sometimes really need sequences from taxa for which sequencing data are particularly poor and a "low-quality" sequence is the best you can do.  In the following pipeline, we include everything up front, and then remove redundant sequences.  We preferentially remove these low-quality sequences if better sequences are available. This is implemented in the jupyter notebooks included in this protocol. 

### Practical thoughts:

+ The bigger the initial dataset, the better. We can pare down later. Do not worry about duplicate sequences at this point; we'll remove them later. 

+ BLAST with multiple paralogs for the gene of interest, sampled from multiple species. 

+ Use PSI-BLAST to iteratively build your dataset.  In PSI-BLAST, your results from the first BLAST search are fed into the next BLAST search, yielding more hits overall.  This process can be repeated until you find no more new hits. 

+ Check for good taxonomic sampling. (You gain more information from 1,000 sequences from across mammals than just taking 1,000 sequences from rodents.) The `Taxonomy` tab on the blast interface is useful for this. If you are not seeing sequences from key species, you can do targeted BLAST within a clade.  

  + To do this, use the `Organism` selection flag to make sure you get sequences from key lineages.  Since we often study vertebrate proteins, here's a useful strategy: do separate BLAST searches for: `Mammalia (taxid:40674)`, `Sauropsida (taxid:8457)`, `Amphibia (taxid:8292)`,  `bony fishes  (taxid:7898)`,  `Elasmobranchii (taxid:7778)` (skates, rays and sharks), `jawless vertebrates (taxid:1476529)` (hagfish and lampreys), and `tunicates (taxid:7712)` (closest non-vertebrate outgroup). I also usually do `lobe-finned fishes (taxid:118072)`, excluding `Tetrapoda (taxid:32523)`.  This should pull up lungfish and coelacanth sequences if available. 
  + If you get a ton of hits in your earliest-diverging lineage--`tunicates` above--it suggests the protein evolved earlier than you have sampled.  If so, expand to earlier-diverging groups.  In this case, you would expand to earlier-diverging groups like `Ecdysozoa (taxid:1206794)` (which includes both *D. Melanogaster* and *C. elegans*), `Lophotrochozoa (taxid:1206795)` and `Hemichordata (taxid:10219)`. 
  + If one of those searches yields a small number of hits, it might be worthwhile to search for non-NCBI genomes and transcriptomes in an effort to fill out the taxa. Some lineages are just have few sequence resources.  Amphibians are notoriously undersampled, and there are very few extant jawless vertebrates. As a result, these bits of vertebrate protein/gene treees are often sparse. 
  
### Other sources of sequences

*Note: If you have to go this route, you'll have to manually load the non-NCBI results into an existing topiary dataframe in excel or the like. See #10 below for an example.*

+ Some good places to look for sequences outside of NCBI are [Fish10K](https://db.cngb.org/datamart/animal/DATAani16/) and [Bird10K](https://b10k.genomics.cn/index.html). You can also switch to [tblastn](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=tblastn&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome) approach to see if there are nucleic acid sequences corresponding to your protein that have not been annotated as proteins.  I find [https://ensembl.org/](https://ensembl.org/) is the easiest database for this task. Finally, if you are desperate, can look for transcriptomes in the [short read archive](https://www.ncbi.nlm.nih.gov/sra).  These often have to be assembled (a somewhat tedious process). 

+ [PFAM](http://pfam.xfam.org/) has some amazing pre-built alignments.  I've generally found them more useful for studies of whole domains evolving over very long timescales than our typical vertebrate protein work, but it's worth keeping in mind these alignments are out there. 


## 2. Load sequences from xml files into a dataframe

This will create a data frame and .csv file with all of the sequences you identified by BLAST.  Most of the columns are self-explanatory, but there are two important columns that bear more explanation: 
 + `uid`: a random 10-letter string unique to each sequence, used for generating files compatible with PAML. 
 + `keep`: whether or not the sequence should be written out in alignments. Sequences are never deleted from the dataframe, just marked as `keep = False`. 
 
Running this code requires you provide a list of `.xml` files.  You can also 

In [ ]:
# List of xml files to load and process
list_of_xml_files = ["example/amphibian.xml"] #,"example/mammalian.xml","example/sauropsid.xml"]

# Aliases as a dictionary. It will map anything in the values to the key. For
# this example, "lymphocyte antigen 96", "MD2", and "MD-2" will all be replaced
# by "LY96". 
alias_dictionary = {"LY96":("lymphocyte antigen 96","MD2","MD-2"),
                    "LY86":("lymphocyte antigen 86","MD1","MD-1")}

# Load sequences into data frame
df = topiary.ncbi_blast_xml_to_df(list_of_xml_files,aliases=alias_dictionary)

# Write output file
df.to_csv("01_initial-hits.csv",index=False)

# Print to notebook
df

## 3. Check sequence identities using reverse BLAST

BLAST can pull down sequences that are homologous, but outside the clade of interest.  (For example, we might want to study TLR4, but BLAST also pulls up TLR2). To identify these sequences, we use reverse BLAST each sequence against the human genome. We will keep only those sequences that pull up the proteins of interest from the human genome. 

The code below will do this reverse BLAST, adding two columns to the data frame: 
 + `rev_hit`: the definition of the top reverse BLAST hit. 
 + `rev_call`: the paralog call based on the reverse BLAST
 
It will also update `keep` to `False` if no reverse hit was found.

`call_dict` determines how calls are made. `rev_blast_db` determines what local database to use for the reverse BLAST.  

In [ ]:
# Read the data frame from the previously written file.  This is not necessary
# if you are running the notebook in order, but is super handy if you want to 
# start the notebook midway through the analysis. 
df = pd.read_csv("01_initial-hits.csv")

# Perform reverse blast, looking for hits on "lymphocyte antigen 96" and 
# "lymphocyte antigen 86" from the human genome, labeling them as LY96 
# and LY86 respectively. 
df = topiary.reverse_blast(df,
                           call_dict={"lymphocyte antigen 96":"LY96",
                                      "lymphocyte antigen 86":"LY86"},
                           rev_blast_db="GRCh38")

# Write output file
df.to_csv("02_reverse-blasted.csv",index=False)

# Print to notebook
df

## 4. Lower redundancy of sequences

To make the computation faster and avoid bias from inclusion of many very similar sequences, we usually remove sequences that are highly similar to one another. The code below will combine sequences with identities greater than 0.9, using relatively intelligent criteria to choose the higher quality sequence. *Note: if you have a lot of sequences, this might initially say it will take forever to run, but each loop gets faster, so it will not take the 40 hrs the progress bar advertises after the first loop.*

The `key_species` list is a list of species that will be given preference over others. If we specify `["Homo sapiens","Mus musculus"]` as key species and then find a human and chimp sequence are highly similar, the software will drop the chimp. If we find a human and a mouse sequence are highly similar, the software will drop whichever sequence has a lower quality. 

In [ ]:
# Read the data frame from the previously written file.  This is not necessary
# if you are running the notebook in order, but is super handy if you want to 
# start the notebook midway through the analysis. 
df = pd.read_csv("02_reverse-blasted.csv")

# Remove redundancy
key_species = ["Homo sapiens","Mus musculus","Monodelphis domestica","Gallus gallus",
               "Xenopus laevis","Danio rerio"]
df = topiary.remove_redundancy(df,0.90,key_species=key_species)

# Write out file
df.to_csv("03_removed-redundancy.csv",index=False)

# Print in notebook
df

## 5. Check and edit reduced data frame

At this point, you might want to look over the set of sequences and see if you like the result of the automatic redundancy reduction.  Some things to look for:

+ Is your sequence set still huge (>1000) or too small (<100)? (If so, you might want to play with the redundnacy cutoff above). 
+ Do you still have the sequence of modern species you care about? (If so, you may want to manually set those sequences to `keep = True`). 

You could load up the nonredudnant set in excel, but I *strongly* recommend you do these manipulations using the pandas dataframe. Pandas slicing allows you to easily pull out rows you care about based on selection criteria. The cell below shows a few of these slicing approaches as templates. 


In [ ]:
# Make a dataframe with only xenopus
df_xenopus = df.loc[df.species == "Xenopus laevis",:]

# Make a dataframe without xenopus
df_not_xenopus = df.loc[df.species != "Xenopus laevis",:]

# Merge two data frames (creating our original data frame)
new_df = pd.concat([df_xenopus,df_not_xenopus],ignore_index=False)

# Set every row with a sequence length less than 100 to False. Note
# we copy the data frame first to avoid editing our main database 
# for this example. 
no_short_df = df.copy()
no_short_df.loc[no_short_df.length < 100,"keep"] = False


## 6. Align the non-redundant sequences using MUSCLE

We now have a database of sequences.  We now need to align those sequences to one another.  The code below will create a file called `04_to-align.fasta` that has all of the sequences flagged with `keep = True`. The sequences will be assigned "pretty" names that have a defined structure: `ortholog_call|species|accession`. 

In [ ]:
# Read the data frame from the previously written file.  This is not necessary
# if you are running the notebook in order, but is super handy if you want to 
# start the notebook midway through the analysis. 
df = pd.read_csv("03_removed-redundancy.csv")

# Write fasta file. 
topiary.write_fasta(df,"04_to-align.fasta",seq_name="pretty")

# Align the sequences in 04_to-align and create 05_aligned
cmd = MuscleCommandline(input="04_to-align.fasta", out="05_aligned.fasta")
stdout, stderr = cmd()

## 7. Manually edit the alignment using AliView

Human brains are still better than computers at identifying patterns in sequence data. We're going to edit the alignment manually. Two consequences of this:

1. We need to publish our final alignment with our manuscript. It needs to be available for evaluation by readers and/or to reproduce the work. 

2.  If we're doing ancestral sequence reconstruction an we *delete* a column, we need to make sure we're okay with not including that column in the final reconstruction.  This probably makes sense for N- and C-terminal extensions, but makes less sense for columns in the middle of the protein. 

With that in mind, open the `05_aligned.fasta` file in aliview and do the following:

1. *Look for sequences that are way longer or shorter than the average.*  Super long sequences may align poorly--and suck other sequences into that poor alignment.  Super short sequences are also difficult to align and provide little taxonomic information.  Delete these sequences by selecting them and going to `Edit->Delete selected`.  When you reimport the alignment, it will set `keep = False` for any sequence you deleted. 

2. *Trim random long N-terminal and C-terminal extensions from alignment.*  Deleting sequences that align poorly will not bias your tree, but including incorrectly aligned regions might. Select these sequence regions and go to `Edit->Clear selected bases`. 

3. *Manually realign problematic regions.*  Sometimes, alignment programs will make obvious mistakes, where the same sequence element is aligned different ways right next to one another.  To correct for this, you can manually move groups of amino acids by selecting them and dragging them right or left. You can also select whole blocks of the alignment and then go to `Align->Realign selected block`. 

4. *Remove empty columns and rows.* Remove gaps-only columns (`Edit->Delete gap-only columns`) and any empty sequences (`Edit->Delete empty sequences`). 

5. *Save out the edited alignment* as:
```
06_aligned-edited.fasta
```

## 8. Load newly aligned sequences into our dataframe and write out .phy file for tree building

We will now load our alignment back into our dataframe.  This will create a new column called "alignment" with the aligned sequences and will also set all sequences *not* in the alignment file to have "keep = False".  We will then write out the aligned sequences into a .phy file, the file format used by RAxML and other tree-inference software packages. 

In [ ]:
# Read the data frame from the previously written file.  This is not necessary
# if you are running the notebook in order, but is super handy if you want to 
# start the notebook midway through the analysis. 
df = pd.read_csv("03_removed-redundancy.csv")

# Load the alignment into the data frame
df = topiary.load_fasta(df,"06_aligned-edited.fasta",load_into_column="alignment")

# Write out file
df.to_csv("07_seq-database_1.0.csv",index=False)

# Write out the alignment in .phy format
topiary.write_phy(df,"08_for-tree-building.phy",seq_column="alignment")

# Print df to notebook
df

## 9. Generate ML phylogenetic tree using RAxML

Our next steps are to find a good evolutionary model that describes our data and to build a maximum likelihoo phylogenetic tree.  This is likely something you will want to run on a high-performance computing cluster.  You need to copy "08_for-tree-building.phy" and "raxml.py" to whatever server you use. Hopefully it already has "raxmlHPC" installed.  You then need to execute two commands, using the helper `raxml.py` script.

The first will search through a collection of different models of evolutionary rate distribution, amino acid substitution probability, etc. and find the model that gives the highest likelihood.  

```
./raxml.py find_model -a 08_for-tree-building.phy -o find_model
```

This will print out the best model at the end, along with an `AIC Prob` score.  Hopefully, that number is close to 1.0, meaning that the chosen model is clearly a better choice than all the others. (If not, we may need to reconstruct our ancestors using different models to make sure the results are robust to the choice of model).  If you want to see the likelihoods and AIC probabilities for all models, check out `find_model/model-comparison.csv`). 

We next need to build the maximum likelihood phylogenetic tree for our alignment. To do that, run the following.  It will automatically grab the best model from the previous calculation. 
```
./raxml.py ml -a 08_for-tree-building.phy -o ml_tree -m `cat find_model/best-model.txt`
```

Once complete, you can download the resulting `ml_tree/final-tree.newick` to your local computer. 

## 10. Evaluate tree

The next step is to look at the ML tree and make sure it is well supported/sensical. To do so, you should first write out the tree with useful names for each taxon. To do so, run the following.

In [ ]:
# Read the data frame from the previously written file.  This is not necessary
# if you are running the notebook in order, but is super handy if you want to 
# start the notebook midway through the analysis. 
df = pd.read_csv("07_seq-database_1.0.csv")

# Make the sequence names on the output tree human readable. 
topiary.util.uid_to_pretty("ml_tree/final-tree.newick","09_ml-tree.newick",df)

Open `09_ml-tree.newick` in FigTree.  On the left-hand panel, go to "Branch-labels" and select "Display: label". This will label each branch with its SH support.  SH support values go from 0 (no support at all) to 100 (excellent support).  Then look at the following:

+ *Are the major clades well supported?* Major branch points should (hopefully) have $SH \ge 85$. If not, we may need to do our reconstructions on multiple versions of the tree to see if our ancestral sequences are robust to the tree topology. 
+ *Is the species tree approximately correct?* Do you see birds with birds, mammals with mammals, etc.?   If not, there could be a problem with the current alignment, or we might need to add more sequences to the alignment. 
+ *Are there long branches?* A long branch is one where you have a bunch of sequence change (say 0.7 subs/site) without branching.  This means the evolutionary model runs and runs without getting input from branches.  This can lead to bias and will certainly lead to very poor reconstructions of ancestors near the long branch. If there is a long branch for a single sequence, delete it from the alignment. It is too divergent or too poorly aligned to include effectively. If a long branch occurs between clades, you can try to find new sequences that "break" the branch.  For example, if there is a long branch between bony fishes and birds, adding amphibian sequences will cut the branch (about) in half and should improve the inference. 

## 11. Iteratively add sequences to alignment

At this point, you may need to go back and add sequences to the alignment. To do so, open `07_seq-database_1.0.csv` in excel and manually add any new sequences to the database. Make sure you fill out every column, including pasting the sequence into the `alignment` column. Save this out as `10_seq-database_1.1.csv`. 

Finally, write this out as a fasta file.  You can then load into aliview, edit, and repeat steps 6-9 until you are satisfied with the ML tree.  (The following code block is an example of what you might run in a jupyter notebook.) 

```
# Read in the manually edited sequence database
df = pd.read_csv("10_seq-database_1.1.csv")

# Write out a fasta file. 
topiary.write_fasta(df,"11_new-alignment.fasta",seq_name="pretty",seq_column="alignment")

### edit in alivew and save as 12_aligned-edited.fasta

# Load the alignment into the data frame
df = topiary.load_fasta(df,"12_aligned-edited.fasta",load_into_column="alignment")

# Write out file
df.to_csv("11_seq-database_1.1.csv",index=False)

# Write out the alignment in .phy format
topiary.write_phy(df,"12_for-tree-building.phy",seq_column="alignment")

```

## 12. Assign orthology

Once you have a tree and alignment that you are happy with, you now need to assign the orthology of each sequence. For this, I will assume this corresponds to `07_seq-database_1.0.csv` and `09_ml-tree.newick`. 

Open the .csv file in excel and add a column called 

## 13. Generate species tree(s)

## 14. Reconstruct ancestors on tree